# 1. Import de Bibliotecas e de Algoritmos

In [1]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [ ]:
import pandas as pd

df= pd.read_csv('dados/train.csv', index_col='id')
df['pricep

# 2. FEATURE ENGINEERING - Extracao e Limpeza de Dados 

In [2]:
def feature_engeneering(df):
    df_eng = df.copy()
    
    # --- LIMPEZA INICIAL ---
    df_eng = df_eng.drop_duplicates()
    df_eng['hp'] = df['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False).astype(float)
    df_eng['liters'] = df['engine'].str.extract(r'(\d+\.?\d*)L\s', expand=False).astype(float)
    
    # --- Idade e Uso ---
    var_ano_atual = date.today().year
    df_eng['car_age'] = var_ano_atual - df_eng['model_year']
    df_eng['car_age'] = df_eng['car_age'].replace(0, 1)
    
    # --- Cilindrada ---
    df_eng['cylinders'] = df['engine'].str.extract(r'(\d+)\s+Cylinder', expand=False)
    df_eng['cylinders'] = df_eng['cylinders'].fillna(df['engine'].str.extract(r'V(\d+)', expand=False))
    df_eng['cylinders'] = df_eng['cylinders'].astype(float)
    
    # --- Tecnologias de Motor ---
    df_eng['is_turbo'] = df['engine'].str.contains(r'(?i)turbo', na=False).astype(int)
    df_eng['turbo_type'] = df['engine'].str.extract(r'(Twin Turbo|Turbo)', expand=False)
    df_eng['valve_train'] = df['engine'].str.extract(r'(DOHC|OHV|SOHC)', expand=False) 
    df_eng['fuel_injection'] = df['engine'].str.extract(r'(PDI|GDI|MPFI)', expand=False)
    
    # Miles per year
    df_eng['miles_p_year'] = df_eng['milage'] / df_eng['car_age']
    
    # --- FUEL TYPE ---
    def clean_fuel(val):
        s = str(val).lower()
        if 'hybrid' in s:
            return 'Hybrid'
        elif 'not supported' in s:
            return 'EV'
        else:
            return val
    df_eng['fuel_type'] = df_eng['fuel_type'].apply(clean_fuel)
    
    # --- TRANSMISSION TYPE ---
    def clean_transmission(val):
        s = str(val).lower()
        if 'automatic' in s or 'a/t' in s or 'cvt' in s:
            return 'Automatico'
        elif 'manual' in s or 'm/t' in s:
            return 'Manual'
        else:
            return 'Outro'
    df_eng['transmission_type'] = df_eng['transmission'].apply(clean_transmission)
    
    # --- Cores ---
    top_ext_colors = df_eng['ext_col'].value_counts().nlargest(10).index
    def simplificar_cor_ext(cor):
        return cor if cor in top_ext_colors else 'Other'
    df_eng['ext_col_simple'] = df_eng['ext_col'].apply(simplificar_cor_ext)
    
    top_int_colors = df_eng['int_col'].value_counts().nlargest(10).index
    def simplificar_cor_int(cor):
        return cor if cor in top_int_colors else 'Other'
    df_eng['int_col_simple'] = df_eng['int_col'].apply(simplificar_cor_int)
    
    # --- Tratamento de Nulos ---
    cols_texto = df_eng.select_dtypes(include=['object']).columns
    df_eng[cols_texto] = df_eng[cols_texto].replace('-', 'Unknown').fillna('Unknown')
    df_eng['clean_title'] = df_eng['clean_title'].replace('Unknown', 'No')
    
    # --- Acidente ---
    def verificar_acidente(valor):
        return 0 if 'None' in str(valor) else 1
    df_eng['accident_clean'] = df_eng['accident'].apply(verificar_acidente)
    
    return df_eng

# 3. PREPARAÇÃO DE DADOS

In [3]:
def preparar_dados(df_treino, df_teste):
    """Prepara dados para modelagem"""
    
    # Aplicar feature engineering
    df_treino_eng = feature_engeneering(df_treino)
    df_teste_eng = feature_engeneering(df_teste)
    
    # Separar target
    y = df_treino_eng['price']
    X = df_treino_eng.drop('price', axis=1)
    X_test = df_teste_eng.copy()
    
    # Selecionar features relevantes
    features_numericas = ['hp', 'liters', 'car_age', 'cylinders', 'miles_p_year', 
                          'milage', 'model_year', 'is_turbo']
    
    features_categoricas = ['brand', 'model', 'fuel_type', 'transmission_type', 
                           'ext_col_simple', 'int_col_simple', 'clean_title', 
                           'turbo_type', 'valve_train', 'fuel_injection']
    
    # Criar dataset numérico
    X_num = X[features_numericas].fillna(0)
    X_test_num = X_test[features_numericas].fillna(0)
    
    # Encodar variáveis categóricas
    X_cat = X[features_categoricas].copy()
    X_test_cat = X_test[features_categoricas].copy()
    
    encoders = {}
    for col in features_categoricas:
        le = LabelEncoder()
        # Fit no treino
        X_cat[col] = X_cat[col].astype(str)
        le.fit(X_cat[col])
        X_cat[col] = le.transform(X_cat[col])
        
        # Transform no teste (tratando categorias novas)
        X_test_cat[col] = X_test_cat[col].astype(str)
        X_test_cat[col] = X_test_cat[col].apply(
            lambda x: le.transform([x])[0] if x in le.classes_ else -1
        )
        encoders[col] = le
    
    # Concatenar features
    X_final = pd.concat([X_num, X_cat], axis=1)
    X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)
    
    return X_final, y, X_test_final, encoders

# 4. DEFINIÇÃO DE MODELOS

In [4]:
def obter_modelos():
    """Retorna dicionário com todos os modelos disponíveis"""
    
    modelos = {
        # Regressão Linear
        'Linear Regression': LinearRegression(),
        
        # K-Nearest Neighbors
        'KNN': KNeighborsRegressor(n_neighbors=5),
        
        # Árvores de Decisão
        'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
        
        # Random Forest e variantes
        'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, 
                                               random_state=42, n_jobs=-1),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, 
                                                       learning_rate=0.1, 
                                                       random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, 
                                    max_depth=7, random_state=42, n_jobs=-1),
        
        # Support Vector Machines
        'SVR Linear': SVR(kernel='linear', C=1.0),
        'SVR RBF': SVR(kernel='rbf', C=1.0, gamma='scale'),
        
        # Redes Neuronais
        'MLP Small': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, 
                                  random_state=42, early_stopping=True),
        
        'MLP Deep': MLPRegressor(hidden_layer_sizes=(100, 50, 25), max_iter=500, 
                                 random_state=42, early_stopping=True)
    }
    
    return modelos

# 5. AVALIAÇÃO DE MODELOS

In [5]:
def avaliar_modelo(modelo, X_train, y_train, X_val, y_val, nome_modelo):
    """Treina e avalia um modelo"""
    
    print(f"\n{'='*60}")
    print(f"Treinando: {nome_modelo}")
    print(f"{'='*60}")
    
    # Treinar
    modelo.fit(X_train, y_train)
    
    # Predições
    y_train_pred = modelo.predict(X_train)
    y_val_pred = modelo.predict(X_val)
    
    # Métricas de treino
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    
    # Métricas de validação
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    val_r2 = r2_score(y_val, y_val_pred)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    # Exibir resultados
    print(f"\n📊 MÉTRICAS DE TREINO:")
    print(f"  RMSE: ${train_rmse:,.2f}")
    print(f"  R²: {train_r2:.4f}")
    print(f"  MAE: ${train_mae:,.2f}")
    
    print(f"\n📊 MÉTRICAS DE VALIDAÇÃO:")
    print(f"  RMSE: ${val_rmse:,.2f}")
    print(f"  R²: {val_r2:.4f}")
    print(f"  MAE: ${val_mae:,.2f}")
    
    # Verificar overfitting
    overfit = train_r2 - val_r2
    if overfit > 0.1:
        print(f"\n⚠️  Possível overfitting detectado (diferença R²: {overfit:.4f})")
    
    return {
        'modelo': nome_modelo,
        'train_rmse': train_rmse,
        'train_r2': train_r2,
        'train_mae': train_mae,
        'val_rmse': val_rmse,
        'val_r2': val_r2,
        'val_mae': val_mae,
        'modelo_treinado': modelo
    }

# 6. TREINAR MODELOS

In [6]:
def treinar_modelos(df_treino, df_teste, modelos_selecionados=None):
    """
    Treina e compara múltiplos modelos
    
    Args:
        df_treino: DataFrame de treino
        df_teste: DataFrame de teste
        modelos_selecionados: Lista com nomes dos modelos a treinar (None = todos)
    """
    
    print("🔄 Preparando dados...")
    X, y, X_test, encoders = preparar_dados(df_treino, df_teste)
    
    # Split treino/validação
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Normalizar dados (importante para KNN, SVM e MLP)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"✅ Dados preparados:")
    print(f"  - Treino: {X_train.shape[0]} amostras")
    print(f"  - Validação: {X_val.shape[0]} amostras")
    print(f"  - Features: {X_train.shape[1]}")
    
    # Obter modelos
    todos_modelos = obter_modelos()
    
    # Filtrar modelos se especificado
    if modelos_selecionados:
        modelos = {k: v for k, v in todos_modelos.items() if k in modelos_selecionados}
    else:
        modelos = todos_modelos
    
    # Treinar modelos
    resultados = []
    modelos_treinados = {}
    
    for nome, modelo in modelos.items():
        # Decidir se usar dados normalizados
        usar_scaled = nome in ['KNN', 'SVR Linear', 'SVR RBF', 'MLP Small', 'MLP Deep']
        
        if usar_scaled:
            res = avaliar_modelo(modelo, X_train_scaled, y_train, 
                                X_val_scaled, y_val, nome)
        else:
            res = avaliar_modelo(modelo, X_train, y_train, 
                                X_val, y_val, nome)
        
        resultados.append(res)
        modelos_treinados[nome] = {
            'modelo': res['modelo_treinado'],
            'usar_scaled': usar_scaled
        }
    
    # Comparação final
    print(f"\n\n{'='*80}")
    print("📈 COMPARAÇÃO DE MODELOS (ordenados por R² de validação)")
    print(f"{'='*80}\n")
    
    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.sort_values('val_r2', ascending=False)
    
    print(df_resultados[['modelo', 'val_rmse', 'val_r2', 'val_mae']].to_string(index=False))
    
    # Melhor modelo
    melhor = df_resultados.iloc[0]
    print(f"\n\n🏆 MELHOR MODELO: {melhor['modelo']}")
    print(f"  R² Validação: {melhor['val_r2']:.4f}")
    print(f"  RMSE Validação: ${melhor['val_rmse']:,.2f}")
    
    return df_resultados, modelos_treinados, scaler, (X_test, X_test_scaled)

# Fim - Execucao do Main

In [ ]:
if __name__ == "__main__":
    # Carregar dados
    df_treino = pd.read_csv('dados/train.csv', index_col='id')
    df_teste = pd.read_csv('dados/test.csv', index_col='id')
    
    # OPÇÃO 1: Treinar todos os modelos
    print("🚀 TREINANDO TODOS OS MODELOS\n")
    resultados, modelos, scaler, dados_teste = treinar_modelos(df_treino, df_teste)
    
    # OPÇÃO 2: Treinar apenas modelos específicos
    # modelos_escolhidos = ['Random Forest', 'XGBoost', 'MLP Deep']
    # resultados, modelos, scaler, dados_teste = treinar_modelos(
    #     df_treino, df_teste, modelos_selecionados=modelos_escolhidos
    # )
    
    # Gerar previsões com o melhor modelo
    melhor_nome = resultados.iloc[0]['modelo']
    melhor_config = modelos[melhor_nome]
    
    X_test, X_test_scaled = dados_teste
    if melhor_config['usar_scaled']:
        previsoes = melhor_config['modelo'].predict(X_test_scaled)
    else:
        previsoes = melhor_config['modelo'].predict(X_test)
    
    # Salvar previsões
    df_submissao = pd.DataFrame({
        'id': df_teste.index,
        'price': previsoes
    })
    df_submissao.to_csv('previsoes.csv', index=False)
    print(f"\n✅ Previsões salvas em 'previsoes.csv'")

🚀 TREINANDO TODOS OS MODELOS

🔄 Preparando dados...
✅ Dados preparados:
  - Treino: 150826 amostras
  - Validação: 37707 amostras
  - Features: 18

Treinando: Linear Regression

📊 MÉTRICAS DE TREINO:
  RMSE: $75,728.95
  R²: 0.1005
  MAE: $23,028.68

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $70,101.47
  R²: 0.1163
  MAE: $23,025.86

Treinando: Ridge

📊 MÉTRICAS DE TREINO:
  RMSE: $75,728.95
  R²: 0.1005
  MAE: $23,028.73

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $70,101.56
  R²: 0.1163
  MAE: $23,025.93

Treinando: Lasso


C:\Users\lucas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.324e+14, tolerance: 9.616e+10
  model = cd_fast.enet_coordinate_descent(



📊 MÉTRICAS DE TREINO:
  RMSE: $75,728.96
  R²: 0.1005
  MAE: $23,028.86

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $70,101.80
  R²: 0.1163
  MAE: $23,026.15

Treinando: KNN

📊 MÉTRICAS DE TREINO:
  RMSE: $65,686.51
  R²: 0.3232
  MAE: $18,200.20

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $76,316.57
  R²: -0.0473
  MAE: $22,587.04

⚠️  Possível overfitting detectado (diferença R²: 0.3705)

Treinando: Decision Tree

📊 MÉTRICAS DE TREINO:
  RMSE: $59,780.73
  R²: 0.4394
  MAE: $18,432.35

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $79,741.27
  R²: -0.1434
  MAE: $20,657.22

⚠️  Possível overfitting detectado (diferença R²: 0.5829)

Treinando: Random Forest

📊 MÉTRICAS DE TREINO:
  RMSE: $39,504.08
  R²: 0.7552
  MAE: $14,973.02

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $72,663.05
  R²: 0.0506
  MAE: $20,737.28

⚠️  Possível overfitting detectado (diferença R²: 0.7046)

Treinando: Gradient Boosting

📊 MÉTRICAS DE TREINO:
  RMSE: $72,855.80
  R²: 0.1674
  MAE: $19,548.80

📊 MÉTRICAS DE VALIDAÇÃO:
  RMSE: $68,276.19
  R²: 0.1